In [7]:
from sklearn import datasets
import numpy as np
import random
%matplotlib inline

In [8]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [144]:
N = X.shape[0]
F = X.shape[1]
epsilon = 0.01
m = 2
c_min = 2 
c_max = 3
C = [i for i in range(c_min, c_max+1)]
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [168]:
def checkMembershipCondition(U):
    return len(U.sum(axis=1)==1)==N;

def evaluateTerminationCriterion(V, U, X, m):
    norm_vals = np.zeros((U.shape[0], U.shape[1]))
    for i in range(len(V)):
        norm_vals[:,i] = np.power(np.linalg.norm(X-V[i], axis=1), 2)
    
    fitness_val = (np.power(U,m)*norm_vals).sum()
    return fitness_val;

def FCM(V_c_i_initial,N,c,m):
    exp_val = -2/(m-1)
    V = np.copy(V_c_i_initial)
    U = np.zeros((N,c))
    iter = 0
    condition = True
    J_n = 1000
    
    while(condition and iter<100):
        iter+=1        
        for i in range(len(V)):
            U[:,i] = np.power(np.linalg.norm(X-V[i],axis=1), exp_val)
        U = U/U.sum(axis=1).reshape(150,-1)
        
        V = []
        for i in range(c):
            col = np.power(U[:,i], m)
            v_i = (col.reshape(N,-1)*X).sum(axis=0)/col.sum()
            V.append(v_i)
        V = np.array(V)
        
        J_n1 = evaluateTerminationCriterion(V,U,X,m)
        condition = (abs(J_n - J_n1) > epsilon) #end when J changes less than 0.001        
        J_n = np.copy(J_n1)
    
    #print('Iterations count for C[i]=',c,'is: ', iter)
    count.append(iter)
    return U, V;

In [142]:
def getDispersion(c,X,V):
    N = X.shape[0]
    F = X.shape[1]
    coeff_variation = X.std(axis=0)/X.mean(axis=0)

    cluster_wise_std = np.zeros((c,F))
    for i in range(len(V)):
        cluster_wise_std[i,:] = np.power(np.power(X-V[i],2).sum(axis=0)/N, 0.5)

    cluster_wise_coeff_variation = cluster_wise_std/V

    Dispersion = cluster_wise_coeff_variation.max()/coeff_variation.max() 
    return Dispersion;

def getSeparation(c, U):
    similarity_matrix = np.zeros((c, c))
    for i in range(c):
        for j in range(i+1,c):
            similarity_matrix[i,j] = np.minimum(U[:,i], U[:,j]).max()
            similarity_matrix[j,i] = np.copy(similarity_matrix[i,j])

    separation_measure = 1 - similarity_matrix
    overall_separation = separation_measure.min()
    return overall_separation;

def getOverlap(c, U):
    N = U.shape[0]
    overlap_matrix = np.zeros((c, c))
    for i in range(c):
        for j in range(i+1,c):
            dom_min = np.minimum(U[:,i],U[:,j])
            dom_max = np.maximum(U[:,i],U[:,j])
            pt_wise_overlap = np.zeros((N,1))

            for k in range(len(dom_min)):
                if(dom_min[k]!=0 and dom_max[k]!=1):
                    if(dom_max[k]>0.5):
                        pt_wise_overlap[k] = 0.9*(1-dom_max[k])*2
                    elif(dom_max[k]):#>=0.1    
                        pt_wise_overlap[k] = 1

            overlap_matrix[i,j] = pt_wise_overlap.sum()
            overlap_matrix[j,i] = np.copy(overlap_matrix[i,j])
    
    total_overlap = overlap_matrix.max()
    return total_overlap;

def getVI_DSO_Row(dispersion_row, separation_row, overlap_row):
    dispersion_row = np.array(dispersion_row)    
    separation_row = np.array(separation_row)
    overlap_row = np.array(overlap_row)

    dispersion_row_normalized = dispersion_row/dispersion_row.max()
    separation_row_normalized = separation_row/separation_row.max()
    overlap_row_normalized = overlap_row/overlap_row.max()
    
    VI_dso_row = (dispersion_row_normalized + overlap_row_normalized)/separation_row_normalized
    return VI_dso_row/VI_dso_row.max()

In [73]:
def initialize_V_best(C, F):
    v_initial = []
    for i in range(len(C)):
        v_initial.append([])
        for j in range(C[i]):
            v_initial[i].append([0 for k in range(F)])    
    return v_initial

def initializeQBitsForAll_C_Centroids(q_bits, C):
    Q_g_matrix = []
    s_g_matrix = []
    
    Q, s = initializeQAndBinaryBits(q_bits)
    for i in range(len(C)):
        V_c_q_bits = []
        V_c_s_bits = []
        for j in range(C[i]):
            V_c_q_bits.append(np.array([np.copy(Q) for k in range(F)]))
            V_c_s_bits.append(np.array([np.copy(s) for k in range(F)]))
        Q_g_matrix.append(V_c_q_bits)
        s_g_matrix.append(V_c_s_bits)
    return Q_g_matrix, s_g_matrix

def callObservationProcessOnClusterCentroidQMatrix(Q_g_matrix, V_c_s_matrix, q_bits, X):
    V_val_matrix = []
    for i in range(len(Q_g_matrix)):
        V_C_g = []
        V_val_matrix.append([])
        for j in range(len(Q_g_matrix[i])):
            V_C_i_g = Q_g_matrix[i][j]
            V_val_matrix[i].append([])
            for dim in range(len(V_C_i_g)):
                val, s_g = observationProcess(V_C_i_g[dim], q_bits, [X[:,dim].min(), X[:,dim].max()])
                V_val_matrix[i][j].append(val)
                V_c_s_matrix[i][j][dim] = s_g
    return V_val_matrix; 

def callUpdateOnClusterCentroidMatrix(Q_g_matrix, F_g_best_C_row, F_g_C, V_c_s_global_matrix, V_c_s_matrix):
    for i in range(len(Q_g_matrix)):
        #V_C_g
        for j in range(len(Q_g_matrix[i])):
            V_C_i_g = Q_g_matrix[i][j]
            for dim in range(len(V_C_i_g)):
                updateQBits(V_C_i_g[dim], F_g_best_C_row[i], F_g_C[i], V_c_s_global_matrix[i][j][dim], V_c_s_matrix[i][j][dim])
    return

In [165]:
def getOptimalClusterNumber(m):
    C = [i for i in range(c_min, c_max+1)]
    q_bits = 2
    V_c_q_matrix, V_c_s_matrix = initializeQBitsForAll_C_Centroids(q_bits, C)
    V_c_s_global_matrix = np.copy(V_c_s_matrix)
    F_g_best_C_row = [1000 for i in range(len(C))]
    F_l_best_C_row = [0 for i in range(len(C))]
    max_generations_for_v_initial = 1
    g = 1

    V_initial_best = initialize_V_best(C, F)
    
    while(g<=max_generations_for_v_initial):
        V_g = []
        F_g_C = []
        dispersion_row = []
        separation_row = []
        overlap_row = []
        V_val_matrix = callObservationProcessOnClusterCentroidQMatrix(V_c_q_matrix, V_c_s_matrix, q_bits, X)

        for i in range(len(C)):
            V_c_i_initial = np.array(V_val_matrix[i])
            U_ci, V_ci = FCM(V_c_i_initial, N, C[i], m)
     
            dispersion_row.append(getDispersion(C[i],X,V_ci))
            separation_row.append(getSeparation(C[i], U_ci))
            overlap_row.append(getOverlap(C[i], U_ci))
        
        F_g_C = getVI_DSO_Row(dispersion_row, separation_row, overlap_row)
        callUpdateOnClusterCentroidMatrix(V_c_q_matrix, F_g_best_C_row, F_g_C, V_c_s_global_matrix, V_c_s_matrix)
        
        for i in range(len(C)):
            if(F_g_C[i]<=F_g_best_C_row[i]):
                F_g_best_C_row[i] = np.copy(F_g_C[i])
                V_c_s_global_matrix[i] = np.copy(V_c_s_matrix[i])
                V_initial_best[i] = np.copy(V_val_matrix[i])
        g+=1
        
    F_g_best_C_row = np.array(F_g_best_C_row)
    VI_dso_min = F_g_best_C_row.min()
    
    index = F_g_best_C_row.argmin()
    optimal_clusters = C[index]
    V_best = V_initial_best[index]
    #print('Optimal Clusters:',optimal_clusters)
    #print('V_initial_best :',V_initial_best)
    return VI_dso_min, optimal_clusters, V_best

In [154]:
VI_dso_min, optimal_clusters, V_best = getOptimalClusterNumber(1.523) #1.529

In [27]:
import math

def observationProcess(Q_g, q_bits, m_range):
    link = 0
    r_g = [random.random() for i in range(q_bits)]
    incr = (m_range[1]-m_range[0])/np.power(2,q_bits)
    sub_spaces = [m_range[0]+(i*incr) for i in range(np.power(2,q_bits)+1)]
    
    s_g = (r_g<=np.power(Q_g,2)).astype(int)
    bin_str = ''.join(s_g.astype(str))
    link = int(bin_str, 2)+1

    selected_sub_space = [sub_spaces[link-1], sub_spaces[link]]
    #count_subspaces.append(link)

    mean_g = (selected_sub_space[0]+selected_sub_space[1])/2

    std_dev_g = (selected_sub_space[1]-mean_g)/4 # so that sub space is covered in -4std to 4std
    m_g = np.random.normal(mean_g, std_dev_g, 1)[0]
    return m_g, s_g;

def getUpdatedQBit(alpha_i, delta_theta):
    updated_val = (alpha_i*math.cos(delta_theta)) - (np.power(1-np.power(alpha_i,2), 0.5)*math.sin(delta_theta))
    
    if(updated_val < np.power(epsilon,0.5)):
        updated_val = np.power(epsilon,0.5)
    elif(updated_val > np.power(1-epsilon,0.5)):
        updated_val = np.power(1-epsilon,0.5)
    
    return updated_val;

def updateQBits(Q_g, F_g_best, F_l_best, s_global, s_g):
    if(F_g_best>F_l_best):
        for i in range(len(s_global)):
            if(s_g[i]==1 and s_global[i]==0):
                delta_theta = 0.03 * math.pi #reduces Q bit value
            else:
                delta_theta = 0
            Q_g[i] = getUpdatedQBit(Q_g[i], delta_theta)    
    else:
        for i in range(len(s_global)):
            if(s_g[i]==0 and s_global[i]==1):
                delta_theta = -0.03 * math.pi #Increases Q bit value
            else:
                delta_theta = 0
            Q_g[i] = getUpdatedQBit(Q_g[i], delta_theta)    
    return Q_g

def initializeQAndBinaryBits(q_bits):
    Q_g = np.array([np.power(q_bits, -0.5) for i in range(q_bits)])
    s_g = np.array([0 for i in range(q_bits)])
    return Q_g, s_g

In [173]:
def EQIE_FCM():
    m_range = [1.5, 2.5]
    q_bits = 2
    Q_m_g, s_m_global = initializeQAndBinaryBits(q_bits)
    F_g_best = 1000
    F_l_best = 0
    max_generations_for_m = 500
    g = 1
    
    while(g<=max_generations_for_m):
        m_g, s_g = observationProcess(Q_m_g, q_bits, m_range)
        VI_dso_min, optimal_clusters, V_initial_best = getOptimalClusterNumber(m_g)
        F_l_best = VI_dso_min
        #xyz.append(F_l_best)
        local_best_params = [np.copy(m_g), np.copy(optimal_clusters)]

        updateQBits(Q_m_g, F_g_best, F_l_best, s_m_global, s_g)

        if(F_g_best > F_l_best):
            global_best_params = np.copy(local_best_params)
            F_g_best = np.copy(F_l_best)
            s_m_global = np.copy(s_g)
        g+=1
        
    return global_best_params;    

In [174]:
count = []
global_best_params = EQIE_FCM()

In [175]:
global_best_params

array([1.525, 2.000])

In [176]:
np.array(count).sum()

8198